In [1]:
# install
!apt update
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# set connection
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres
!wget https://users.dcc.uchile.cl/~mtoro/cursos/BD2020/adult_with_pii.csv

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [101 kB]
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:13 https://developer.download.nvidia.com/comp

In [2]:
%%sql
DROP TABLE IF EXISTS person;
CREATE TABLE person
(
  name varchar(255),
  dob date,
  ssn  varchar(50),
  zip integer,
  age integer,
  workclass varchar(255),
  fnlwgt integer,
  education varchar(255),
  education_num integer,
  marital_status varchar(255),
  occupation varchar(255),
  relationship varchar(255),
  race varchar(255),
  sex varchar(50),
  capital_gain integer,
  capital_loss integer,
  hours_per_week integer,
  country varchar(255),
  target varchar(255)
);
ALTER TABLE ONLY person ADD CONSTRAINT "person_key" PRIMARY KEY (name,ssn);

 * postgresql+psycopg2://@/postgres


""


Suponga que el gobierno de Estados Unidos cuenta con un base de datos de resultados del censo (la información de identificación personal (PII) es inventada)

In [3]:
%%sql
copy person(name, dob, ssn, zip, age, workclass, fnlwgt, education, education_num, marital_status, occupation, relationship, race, sex, capital_gain, capital_loss, hours_per_week, country, target) from
'/content/adult_with_pii.csv' delimiter ',' csv header;

 * postgresql+psycopg2://@/postgres


""


In [4]:
%sql SELECT * FROM person limit 5;

 * postgresql+psycopg2://@/postgres


,name,dob,ssn,zip,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,country,target
0,Karrie Trusslove,1967-09-07,732-14-6110,64152,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,Brandise Tripony,1988-06-07,150-19-2766,61523,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,Brenn McNeely,1991-08-06,725-59-9860,95668,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,Dorry Poter,2009-04-06,659-57-4974,25503,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,Dick Honnan,1951-09-16,220-93-3811,75387,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Suponga que los identificadores explícitos de las personas son las columnas name y ssn (social security number). 
Para proteger la privacidad de las personas, el gobierno de-identifica esta tabla borrando las columnas name y ssn.

In [ ]:
%%sql 
create view person_deid as select dob, zip, age, workclass, fnlwgt, education, education_num, marital_status, occupation, relationship, race, sex, capital_gain, capital_loss, hours_per_week, country, target from person; 

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql select * from person_deid limit 5;

 * postgresql+psycopg2://@/postgres


,dob,zip,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,country,target
0,1967-09-07,64152,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,1988-06-07,61523,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,1991-08-06,95668,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,2009-04-06,25503,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,1951-09-16,75387,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Suponga también que se filtró de una casa comercial, los nombres, fechas de nacimiento, ssn, código postal y edad de muchas personas. Nadie se preocupa por esto ya que no contiene información sensible (como el sueldo).
Crearemos una vista para representar esta data auxiliar.

In [ ]:
%%sql 
create view aux as select name, dob, ssn, zip, age from person; 

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql select * from aux limit 5;

 * postgresql+psycopg2://@/postgres


,name,dob,ssn,zip,age
0,Karrie Trusslove,1967-09-07,732-14-6110,64152,39
1,Brandise Tripony,1988-06-07,150-19-2766,61523,50
2,Brenn McNeely,1991-08-06,725-59-9860,95668,38
3,Dorry Poter,2009-04-06,659-57-4974,25503,53
4,Dick Honnan,1951-09-16,220-93-3811,75387,28


#Pregunta 1

Escriba una función que tome como argumento un conjunto de columnas y que realice un ataque de asociación a la tabla person_deid usando la tabla aux.
Muestre el resultado de al menos 3 ataques de asociación.


#  Pregunta 2

1.   Escriba cuántos individuos en esta base de datos son únicamente identificados por el código postal (zip). ¿A qué porcentaje del total corresponde?
2.   Haga lo mismo para el CId edad (age).
3.   ¿Qué puede concluir de este análisis?




# Pregunta 3
El gobierno se da cuenta de que su intento de anonimización es muy vulnerable, por lo que solo permite a los analistas a realizar consultas de agregación sobre la tabla person que involucren muchas filas. 
Obtenga el nivel educativo (education_num) del individuo llamado Ardyce Golby usando solo consultas de agregación que involucren al menos 1000 tuplas sobre la tabla *person*.

#Pregunta 4


1.   Suponga ahora que un atacante cuenta con data auxiliar que contiene el cuasi-identificador código postal (zip). Indique qué nivel de $k$-Anonimato cumple person_deid.
2.   Utilice alguna técnica de generalización y/o eliminación de valores atípicos sobre los cuasi-identificadores para lograr al menos $2$-anonimato.
3.   Repita los pasos 1 y 2 para el CId edad (age).
4.   Repita los pasos 1 y 2 para los CIds (zip, age). *Hint*: puede tratar de eliminar valores atípicos para la columna age.
5.   Concluya qué es lo que pasa cuando agregamos más CIds al análisis de $k$-anonimato.




#Pregunta 5

Suponga que los CIds son edad y nivel de educación (age, education_num), y estado civil un dato sensible (marital_status).

1.   Escriba una consulta e indique si la tabla es $2$-diversa.
2.   Utilice generalización y/o eliminación de valores atípicos para lograr al menos $5$-diversidad.
3.   Para el resultado en (2), escriba la consulta e indique el nivel de $k$-anonimato que se logra. ¿Qué puede concluir de esto?
4.   (Opcional/Bonus) La **diversidad entrópica** de un grupo con el mismo CIds mide cuán uniformemente están distribuidos los valores privados de algún atributo $S$ en el grupo. Se define formalmente como
$$-\sum_{v\in S}\mathtt{P}(\mathit{cIds}, v)*\mathtt{log}(\mathtt{P}(\mathit{cIds}, v))$$ donde $\mathtt{P}(\mathit{cIds}, v)$ corresponde a la fracción de registros en el grupo $\mathit{cIds}$ con valor $v$ para el atributo $S$.
Por ejemplo, para la siguiente tabla donde la enfermedad es un dato sensible:

| Trabajo | Sexo | Edad | Enfermedad |
| --- | --- | --- | --- |
| Profesor | Masculino | [35-40) | Hepatitis |
| Profesor | Masculino | [35-40) | Hepatitis |
| Profesor | Masculino | [35-40) | VIH |
| Artista | Femenino | [35-35) | Resfrío |
| Artista | Femenino | [35-35) | VIH |
| Artista | Femenino | [35-35) | VIH |
| Artista | Femenino | [35-35) | VIH |

La diversidad entrópica del grupo (Profesor, Masculino [35-40)) es de $-(\frac{2}{3}\mathtt{log}\frac{2}{3} + \frac{1}{3}\mathtt{log}\frac{1}{3}) = \mathtt{log}(1.9)$, y la diversidad entropica del grupo (Artista, Femenino, [35-35)) es $-(\frac{3}{4}\mathtt{log}\frac{3}{4} + \frac{1}{4}\mathtt{log}\frac{1}{4}) = \mathtt{log}(1.8)$.

Calcule la **diversidad entrópica** de cada grupo en (2), y concluya cuál es el conjunto de cids más y menos diversos.



# Pregunta 6

1.   Escriba una consulta que responda a "¿cuántas personas nunca se han casado?"
2.   La sensibilidad de una consulta se puede calcular como: cuánto es lo máximo que puede variar el resultado de mi consulta si saco/agrego un único individuo de la base de datos. Escriba y justifique la sensibilidad de la consulta de (1).
3.   Use la distribución de laplace para contestar (1) de manera diferencialmente privada, con epsilon = 0.1. 
    
    *Hint*: Para generar ruido aleatorio puede utilizar la función laplace de numpy:
```
import numpy as np
np.random.laplace(loc=0, scale=s)
```
y para aplicar una función a todos los elementos de un resultado de un consulta (dataframe) puede usar la función apply:
```
r = %sql ...
r.apply(lambda x: ...)
```
4.   Escriba una consulta $0.1$-diferencialmente privada que calcule la cantidad de personas agrupadas por edad y estado civil. Asegúrese que su respuesta no contenga cantidades negativas ni decimales (puede modificar los valores después de agregarle ruido).

    *Hint*: Use la función map para aplicar una función a todos los elementos de una columna de la siguiente forma:
```
r = %sql ...
r['nombre_columna_nueva'] = r['nombre_columna'].map(lambda x: ...)
```

5.    Comente acerca de la utilidad y la privadcidad de la respuesta para grupos muy chicos vs grupos grandes después de agregar ruido. 
